In [20]:
import sqlite3 as sq
import pandas as pd
import numpy as np
import os

from pandas import HDFStore, DataFrame

## read data from file

In [53]:
df      = pd.read_csv('XETRA.csv')
df      = df.drop_duplicates(subset='ISIN',keep='first')
ctuples = [tuple(x) for x in df.values]

In [5]:
class Balance:
    # morning star balance is always in millions
    
    def __init__(self):
        self.currency = 'NON'
    
    def get_units(self):
        self.currency = self.data.columns[0].split('.')[1].split()[0]
        self.factors  = b.data.columns[0].split('in')[-1].split()[0]
        
    def readBalance(self,filename):
        self.data = pd.read_csv(filename,skiprows=1)
        self.get_units()

[('DP WORLD LTD    DL 2', 'AEDFXA0M6V00', '3DW'),
 ('SINOVAC BIOTECH   DL-,001', 'AGP8696W1045', 'SVQ'),
 ('SCHLUMBERGER   DL-,01', 'AN8068571086', 'SCL'),
 ('HUNTER DOUGLAS      EO-24', 'ANN4327C1220', 'HUD'),
 ('ORTHOFIX INT.      DL-,10', 'ANN6748L1027', 'OFX'),
 ('SAPIENS INTL NV    DL-,01', 'ANN7716A1513', 'SPS1'),
 ('BBVA BCO FR.         AP 1', 'ARP125991090', 'BDPA'),
 ('STRABAG SE', 'AT000000STR1', 'XD4'),
 ('AMAG AUSTRIA METALL INH.', 'AT00000AMAG3', 'AM8'),
 ('FACC AG INH.AKT.', 'AT00000FACC2', '1FC'),
 ('FLUGHAFEN WIEN AG', 'AT00000VIE62', 'FLW1'),
 ('RAIFF.-TOPDIVID.-AKT.R A', 'AT0000495288', 'RKC3'),
 ('SCHOELLERBK ZINSSTR.PL. A', 'AT0000497409', 'J7NE'),
 ('AGRANA BET.AG INH.', 'AT0000603709', 'AGB1'),
 ('RAIFFEISEN BK INTL INH.', 'AT0000606306', 'RAW'),
 ('PORR AG', 'AT0000609607', 'ABS2'),
 ('SCHOELLERBK AKT.WAEHRG.T', 'AT0000612692', 'J7N0'),
 ('C-QUADRAT INVESTM.AG EO 1', 'AT0000613005', 'C8I'),
 ('RAIFF.EMERGINMARKE.RA ST.', 'AT0000636733', 'RKCC'),
 ('RAIFF.-EUR.-SM

test the class on a simple example

In [6]:
b = Balance()
b.readBalance('XETRA_ms_balance/R6C3-ROYAL-DUTCH-SHELL-B-EO-07.csv')
b.data.columns = ['Entity',2011,2012,2013,2014,2015]


In [7]:
b_categories = b.data.Entity.values.tolist()

In [8]:
for i,v in enumerate(b.data.Entity.values):
    
    categories = ['Current assets','Non-current assets',"Liabilities and stockholders' equity", 
                  "Current liabilities"]
    
    if v in categories:
        print v, i

Current assets 1
Non-current assets 10
Liabilities and stockholders' equity 23
Current liabilities 25


## read the different possible currencies

In [29]:
currencies = []
factors    = []

for filename in os.listdir('XETRA_ms_balance/'):
    if filename.endswith(".csv"): 
        try:
            b = Balance()
            b.readBalance('XETRA_ms_balance/'+filename)
            currencies.append(b.currency)
            factors.append(b.factors)
        except:
            pass
#         print filename
    else:
        continue
        
currencies = np.array(currencies)
factors    = np.array(factors)

In [30]:
# list of all currencies
currencies = np.unique(currencies)

In [31]:
currencies

array(['AED', 'ARS', 'AUD', 'BDT', 'BGN', 'BRL', 'CAD', 'CHF', 'CNY',
       'COP', 'CZK', 'DKK', 'EGP', 'EUR', 'GBP', 'GEL', 'GHS', 'HKD',
       'HUF', 'IDR', 'ILS', 'INR', 'JMD', 'JPY', 'KRW', 'KWD', 'KZT',
       'MAD', 'MXN', 'MYR', 'NGN', 'NOK', 'NZD', 'OMR', 'PEN', 'PHP',
       'PKR', 'PLN', 'RON', 'RUB', 'SAR', 'SEK', 'SGD', 'THB', 'TRY',
       'TWD', 'UAH', 'USD', 'VND', 'ZAR'], 
      dtype='|S3')

In [34]:
np.unique(factors)

array(['millions'], 
      dtype='|S8')

In [13]:
hdf = HDFStore('database/dfs.h5')

In [14]:
currencies = pd.DataFrame(currencies)

In [16]:
currencies.columns = ['currency']

In [17]:
currencies 

,currency
0,AED
1,ARS
2,AUD
3,BDT
4,BGN
5,BRL
6,CAD
7,CHF
8,CNY
9,COP


In [18]:
hdf.put('currencies',currencies,format='table',data_columns=True)

In [19]:
hdf.close()

Populate sql table

In [42]:
tupels = []
for c in currencies:
    tupels.append(((c,)))

In [45]:
conn = sq.connect('database/stockDB.db')
c    = conn.cursor()

c.execute('''CREATE TABLE IF NOT EXISTS currencies 
    (
    currency    TEXT PRIMARY KEY
    );''')

c.executemany('INSERT INTO currencies VALUES (?)', tupels)

conn.commit()
conn.close()

## Possible entries in Equity

In [8]:
def show_entries_in_category(startline,endline,nmax=0,silent=False):
    pline    = False
    entities = []
    n        = 1
    
    #print startline
    #print endline
    
    for filename in os.listdir('XETRA_ms_balance/'):
        if filename.endswith(".csv"):
#             print filename
            n+=1
            if n==nmax:
                break
            try:
#             if True:
                with open('XETRA_ms_balance/'+filename) as f:
                    pline = False
                    for line in f:
#                         print line
                        if line==endline+"\n":
                            pline = False
                            
                        if pline and line.split(',')[0].replace('\n','').replace('"','') not in entities:
                            if not silent:
                                print line.split(',')[0]        
                            entities.append(line.split(',')[0].replace('\n','').replace('"',''))
                            
                        if line==startline+"\n":
                            pline=True
            except:
                pass
        else:
            continue    
    
    return np.array(entities)

In [9]:
balance_entries = {}

In [10]:
balance_entries['Current Assets'] = show_entries_in_category("Current assets",
                                                             "Non-current assets",
                                                             nmax=0,
                                                             silent=True)

In [11]:
balance_entries['Non-Current Assets'] = show_entries_in_category("Non-current assets",
                                                                 "Liabilities and stockholders' equity",
                                                                 nmax=0,
                                                                 silent=True)

In [12]:
balance_entries['Stockholders Equity'] = show_entries_in_category("Stockholders' equity","---",
                                                                  nmax=0,
                                                                  silent=True)

In [13]:
balance_entries['Current liabilities'] = show_entries_in_category("Current liabilities",
                                                                  "Non-current liabilities",
                                                                  silent=True)

In [14]:
balance_entries['Non-current liabilities'] = show_entries_in_category("Non-current liabilities","",silent=True)

## Define Categories and sub-categories

In [15]:


balance_entries['Non-current liabilities']

array(['Long-term debt', 'Deferred taxes liabilities', 'Minority interest',
       'Other long-term liabilities', 'Total non-current liabilities',
       'Total liabilities', "Stockholders' equity", 'Preferred stock',
       'Common stock', 'Additional paid-in capital', 'Retained earnings',
       'Treasury stock', 'Accumulated other comprehensive income',
       "Total stockholders' equity",
       "Total liabilities and stockholders' equity", 'Capital leases',
       "Total Stockholders' equity", 'Deferred tax liabilities',
       'Pensions and other benefits', 'Accrued liabilities',
       'Deferred revenues', 'Pensions and other postretirement benefits',
       'Regulatory liabilities', "stockholders' equity", 'General Partner',
       'Other Equity'], 
      dtype='|S42')

In [2]:
class BalanceSQL:
    
    def __init__(self,filename):
        self.filename = filename
        self.currency = 'NON'
        self.factor   = 'ONE'
        self.catids   = pd.DataFrame(columns=["catid","category","subcategory"])     
        pass

    def check_filename(self):
        if self.filename=='Undefined':
            print 'filename not defined. please indicate filename'
            
    def get_isin(self):
        self.ticker = self.filename.split('/')[-1]
        self.ticker = self.ticker.split('-')[0]
        
        self.name,self.isin,self.ticker = np.array(ctuples)[np.array(ctuples)[:,2]==self.ticker][0]
        pass

    
    def get_header(self):
        self.check_filename()
        
        with open(self.filename) as f:
            for line in f:        
                if 'Fiscal' in line:
                    
                    self.years = line.replace('\n','').split(',')[1:]                
                    self.years = [int(y.split('-')[0]) for y in self.years]
                    
                    self.currency = line.split('.')[1].split()[0]
                    self.factor   = line.split('in')[-1].split()[0]
                    
        if self.factor == 'millions':
            self.factor=1e6

    def return_SQL_tupel_from_line(self,line,category,subcategory,catid,year):
                
        line      = line.replace('property,','property').replace('"','')
        line      = line.replace('Property,','Property').replace('"','')
                
        line      = line.replace('\n','').split(',')
        
        entryname = line[0] 
        values    = line[1:]
        
        year_indx = np.where(np.array(self.years)==year)[0][0]
        
        try:
            values    = np.array(values).astype('float')
        except:
            for i,v in enumerate(values):
                if v=='':
                    values[i] = 0
                else:
                    values[i] = int(v)
                    
        values = np.array(values)
        return  self.isin, year, catid, entryname, values[year_indx]       
    
    
    ######################################################################################################
    # HANDLING OF ENTRY IDS
    ######################################################################################################
    
    def assign_entry_ID(self,line,category,subcategory):
        '''Assign an id for the respective entry. This function should only be used once to initialize
        the SQL database and define all types of entry'''
        
        line      = line.replace('property,','property').replace('"','')
        line      = line.replace('Property,','Property').replace('"','')
        line      = line.replace('\n','').split(',')
        
        entryname = line[0]     
        return  category, subcategory, entryname      
    
        
    def get_all_entry_IDs(self):
                
        category    = "None"
        subcategory = "None"
        entryname   = "None"
        catid       = "None"
        
        cols = ["category","subcategory","entry"]
        cats = pd.DataFrame(columns=cols)
        
        with open(self.filename) as f:    
            for line in f:

                category, subcategory, catid = self.get_category(line,category,subcategory,catid)
                
                if "Fiscal year" in line or "BALANCE SHEET" in line:
                    continue
                
                line      = line.replace('property,','property').replace('"','')
                line      = line.replace('Property,','Property').replace('"','')
                line      = line.replace('"','')  
                line      = line.replace("'","")
                line      = line.replace('\n','').split(',')
                entryname = line[0]  
                
                cats = cats.append(pd.DataFrame([[category,subcategory,entryname]],
                                                 columns=cols),ignore_index=True)
        
        return cats
    
    
    
    def get_category(self,line,ini_category,ini_subcategory,ini_catid):
        
        category    = "None"                           # the category
        subcategory = "None"                           # the subcategory 
        catid       = "None"                           # this is the id used in the SQL table
        
        if line=="Assets\n":
            category    = "Assets"
            subcategory = "Assets"
            catid       = "A01"
            
        if line=="Current assets\n":
            category    = "Assets"
            subcategory = "Current assets"   
            catid       = "A02"

        if line=="Non-current assets\n":
            category    = "Assets"
            subcategory = "Non-current assets"    
            catid       = "A03"
            
        if "Total assets" in line:
            category    = "Assets"
            subcategory = "Total assets"
            catid       = "A04"
            
        if line=="Liabilities\n":
            category    = "Liabilities and equity"
            subcategory = "Liabilities"
            catid       = "L01"
            
        if line=="Current liabilities\n":
            category    = "Liabilities and equity"
            subcategory = "Current liabilities"
            catid       = "L02"

        if line=="Non-current liabilities\n":
            category    = "Liabilities and equity"
            subcategory = "Non-current liabilities"
            catid       = "L03"
            
        if "Total liabilities" in line and 'equity' not in line:
            category    = "Liabilities and equity"
            subcategory = "Total liabilities"             
            catid       = "L04"

        if line=="Stockholders' equity\n":
            category    = "Liabilities and equity"
            subcategory = "Equity"
            catid       = "L05"

        if "Total liabilities and stockholders' equity" in line:
            category    = "Total liabilities and equity"
            subcategory = "Total liabilities and equity"  
            catid       = "L06"
                   
        catidentry = pd.DataFrame(np.array([[catid,category, subcategory]]),
                     columns=["catid","category","subcategory"])
        
        
        if category!="None" and subcategory!="None":
            self.catids = self.catids.append(catidentry,ignore_index=True)
            return category, subcategory, catid
        else:
            return ini_category, ini_subcategory, ini_catid
    
    
    def get_sql_balance_input(self,year,silent=False,show_sql=False):

        self.get_header()        
        
        balanceitems = []
        
        category    = "None"
        subcategory = "None"    
        catid       = "None"
        
        self.get_isin()
        
        if show_sql:
            print '-->  SQL BALANCE INFORMATION'
            print '-->  Currency  ', self.currency
            print '-->  Factor    ', self.factor
            print '-->  Year      ', year
            print '-->  Name      ', self.name
            print '-->  ISIN      ', self.isin
            print '-->  Ticker    ', self.ticker
            print
            
        self.stocks_entries = (self.isin, self.name, self.ticker, self.factor, self.currency)
#         self.balance_infos  = (self.isin, self.year, self.factor)
            
        with open(self.filename) as f:
            
            for line in f:

                category, subcategory, catid = self.get_category(line,category,subcategory,catid)
                    
                if category!="None" and line !='':

                    line = line.replace('property,','property')              # correct for some inconsistencies
                    line = line.replace('Property,','Property')
                    line = line.replace('"','')
                    
                    
                    if len(line.split(','))>1:
                        tupel = self.return_SQL_tupel_from_line(line,category,subcategory,catid,year)
                        balanceitems.append(tupel)
                        if show_sql:
                            print tupel 
                            
                    line = line.replace('property,','property').replace('"','')
                    line = line.replace("'","")

                    try:
                        if not silent:
                            print '%29s  %29s  %42s  %10i'  % (category, subcategory,line.split(',')[0],
                                                                    int(line.split(',')[1]))
                    except:
                        pass
                    
                    
                if 'Total assets' in line and not silent:
                    print 

        self.balance_entries = balanceitems


## Examples for single stocks

In [113]:
RDSB = BalanceSQL('XETRA_ms_balance/R6C3-ROYAL-DUTCH-SHELL-B-EO-07.csv')
RDSB.get_sql_balance_input(2013,silent=True,show_sql=False)

In [114]:
MUV2 = BalanceSQL('XETRA_ms_balance/MUV2-MUENCH.RUECKVERS.VNA-O.N..csv')
# MUV2.get_sql_balance_input(2013,silent=True,show_sql=False)

In [3]:
MUV2 = BalanceSQL('XETRA_ms_balance/MUV2-MUENCH.RUECKVERS.VNA-O.N..csv')
MUV2.get_all_entry_IDs()

,category,subcategory,entry
0,Assets,Assets,Assets
1,Assets,Assets,Cash and cash equivalents
2,Assets,Assets,Deferred policy acquisition costs
3,Assets,Assets,Property and equipment
4,Assets,Assets,Goodwill
5,Assets,Assets,Other assets
6,Assets,Total assets,Total assets
7,Assets,Total assets,Liabilities and stockholders equity
8,Liabilities and equity,Liabilities,Liabilities
9,Liabilities and equity,Liabilities,Unearned premiums


 ## Define category labels

In [4]:
# find all possible categories

cols       = ["category","subcategory","entry"]
categories = pd.DataFrame(columns=cols)

for i,filename in enumerate(os.listdir('XETRA_ms_balance/')):
    if filename.endswith(".csv"):
        try:
            d          = BalanceSQL('XETRA_ms_balance/'+filename)
            cats       = d.get_all_entry_IDs()
            categories = categories.append(cats,ignore_index=True)
            categories.drop_duplicates(inplace=True, keep='last')
        except:
            pass
        

In [29]:
# label all possible categories 

catlabels = []

for index, row in categories.iterrows():
    
    for j in range(len(categories)):
        ii = '%02i' % j
        label = row['category'][0]+str(ii)
        if label not in catlabels:
            catlabels.append(label)
            break

categories = categories.assign(catlabel=np.array(catlabels))

In [34]:
for index, row in categories.iterrows():
    print (row['catlabel'], row['category'], row['subcategory'], row['entry'])

('L00', 'Liabilities and equity', 'Total liabilities', 'Other Equity')
('A00', 'Assets', 'Assets', 'Foreclosed real estate')
('L01', 'Liabilities and equity', 'Current liabilities', 'Dividends payable')
('A01', 'Assets', 'Assets', 'Securities purchased under resale agreements')
('A02', 'Assets', 'Assets', 'Cash')
('A03', 'Assets', 'Assets', 'Total cash and cash equivalents')
('A04', 'Assets', 'Assets', 'Securities')
('A05', 'Assets', 'Assets', 'Trading account securities')
('A06', 'Assets', 'Assets', 'Securities available for sale')
('A07', 'Assets', 'Assets', 'Securities held to maturity')
('A08', 'Assets', 'Assets', 'Federal Home Loan Bank stock')
('A09', 'Assets', 'Assets', 'Total securities')
('A10', 'Assets', 'Assets', 'Commercial loans')
('A11', 'Assets', 'Assets', 'Residential loans')
('A12', 'Assets', 'Assets', 'Consumer loans')
('A13', 'Assets', 'Assets', 'Total loans')
('A14', 'Assets', 'Assets', 'Interest receivable')
('A15', 'Assets', 'Assets', 'Other intangibles')
('L02', 

In [81]:
MUV2.stocks_entries

('DE0008430026', 'MUENCH.RUECKVERS.VNA O.N.', 'MUV2', 1000000.0, 'EUR')

In [82]:
for j in MUV2.balance_entries:
    print j

('DE0008430026', 2013, 'A01', 'Cash and cash equivalents', 2820.0)
('DE0008430026', 2013, 'A01', 'Deferred policy acquisition costs', 9542)
('DE0008430026', 2013, 'A01', 'Property and equipment', 3762.0)
('DE0008430026', 2013, 'A01', 'Goodwill', 4672.0)
('DE0008430026', 2013, 'A01', 'Other assets', 233492.0)
('DE0008430026', 2013, 'A04', 'Total assets', 254288.0)
('DE0008430026', 2013, 'L01', 'Unearned premiums', 7994.0)
('DE0008430026', 2013, 'L01', 'Long-term debt', 248.0)
('DE0008430026', 2013, 'L01', 'Deferred income taxes', 8188)
('DE0008430026', 2013, 'L01', 'Taxes payable', 2795.0)
('DE0008430026', 2013, 'L01', 'Other liabilities', 209080.0)
('DE0008430026', 2013, 'L04', 'Total liabilities', 228305.0)
('DE0008430026', 2013, 'L05', 'Retained earnings', 16188.0)
('DE0008430026', 2013, 'L05', 'Accumulated other comprehensive income', 9795.0)
('DE0008430026', 2013, 'L05', "Total stockholders' equity", 25983.0)
('DE0008430026', 2013, 'L06', "Total liabilities and stockholders' equity

## Populate SQL Database with Balance Data

In [35]:
conn = sq.connect('database/stockDB.db')
c    = conn.cursor()

c.execute('''CREATE TABLE categories 
    (
    catlabel    TEXT PRIMARY KEY,    
    category    TEXT NOT NULL,
    subcategory TEXT NOT NULL,
    entry       TEXT NOT NULL
    );''')


tupels = []
for index, row in categories.iterrows():
    tupels.append((row['catlabel'], row['category'], row['subcategory'], row['entry']))

c.executemany('INSERT INTO categories VALUES (?,?,?,?)', tupels)

conn.commit()
conn.close()

In [57]:
ctuples

[('DP WORLD LTD    DL 2', 'AEDFXA0M6V00', '3DW'),
 ('SINOVAC BIOTECH   DL-,001', 'AGP8696W1045', 'SVQ'),
 ('SCHLUMBERGER   DL-,01', 'AN8068571086', 'SCL'),
 ('HUNTER DOUGLAS      EO-24', 'ANN4327C1220', 'HUD'),
 ('ORTHOFIX INT.      DL-,10', 'ANN6748L1027', 'OFX'),
 ('SAPIENS INTL NV    DL-,01', 'ANN7716A1513', 'SPS1'),
 ('BBVA BCO FR.         AP 1', 'ARP125991090', 'BDPA'),
 ('STRABAG SE', 'AT000000STR1', 'XD4'),
 ('AMAG AUSTRIA METALL INH.', 'AT00000AMAG3', 'AM8'),
 ('FACC AG INH.AKT.', 'AT00000FACC2', '1FC'),
 ('FLUGHAFEN WIEN AG', 'AT00000VIE62', 'FLW1'),
 ('RAIFF.-TOPDIVID.-AKT.R A', 'AT0000495288', 'RKC3'),
 ('SCHOELLERBK ZINSSTR.PL. A', 'AT0000497409', 'J7NE'),
 ('AGRANA BET.AG INH.', 'AT0000603709', 'AGB1'),
 ('RAIFFEISEN BK INTL INH.', 'AT0000606306', 'RAW'),
 ('PORR AG', 'AT0000609607', 'ABS2'),
 ('SCHOELLERBK AKT.WAEHRG.T', 'AT0000612692', 'J7N0'),
 ('C-QUADRAT INVESTM.AG EO 1', 'AT0000613005', 'C8I'),
 ('RAIFF.EMERGINMARKE.RA ST.', 'AT0000636733', 'RKCC'),
 ('RAIFF.-EUR.-SM

In [59]:
conn = sq.connect('database/stockDB.db')
c    = conn.cursor()

c.execute('''CREATE TABLE IF NOT EXISTS stocks 
    (
    NAME    TEXT,
    ISIN    CHAR(12) PRIMARY KEY,
    TICKER  TEXT
    );''')

c.executemany('INSERT INTO stocks VALUES (?,?,?)', ctuples)

# c.executemany('INSERT INTO stocks (`NAME`, `ISIN`, `TICKER`) VALUES (?,?,?) ON DUPLICATE KEY UPDATE `price` = 22.5;')

conn.commit()
conn.close()

In [247]:
failed = []

conn = sq.connect('database/stockDB.db')
c    = conn.cursor()

c.execute('''CREATE TABLE balance 
    (
    isin        TEXT PRIMARY KEY,    
    year        INT  NOT NULL,
    catid       TEXT NOT NULL,
    entity      TEXT NOT NULL,
    amount      REAL NOT NULL
    );''')

c.execute('''CREATE TABLE stocks 
    (
    isin        TEXT PRIMARY KEY,    
    name        TEXT NOT NULL,
    ticker      TEXT NOT NULL,
    factor      REAL,
    currency    TEXT
    );''')


c.execute('''CREATE TABLE categories 
    (
    isin        TEXT PRIMARY KEY,    
    name        TEXT NOT NULL,
    ticker      TEXT NOT NULL,
    factor      REAL,
    currency    TEXT
    );''')




for filename in os.listdir('XETRA_ms_balance/'):
    if filename.endswith(".csv"): 
        b = BalanceSQL()
        b.filename = 'XETRA_ms_balance/'+filename
        b.get_header()
        try:
            for y in b.years:
                tupels = b.return_balance_with_categories(y,silent=True,show_sql=False)
                c.executemany('INSERT INTO balance VALUES (?,?,?,?,?,?,?,?,?)', tupels)
        except:
            failed.append(filename)

conn.commit()
conn.close()



## LOAD KEYRATIOS

In [76]:
class keyratios:
    def __init__(self,ticker,isin='',directory=''):
        filename  = directory + ticker+'.csv'
        self.ticker = ticker
        self.isin   = isin
        self.keyr = pd.read_csv(filename,skiprows=2)
        self._correct_colnames()
    
    def _correct_colnames(self):
        newcols = []
        for i in self.keyr.columns:
            if i == 'TTM':
                newcols.append(newcols[-1]+1)
                continue
            if 'Unnamed' in i:
                newcols.append(0)
                continue 
            newcols.append(int(i.split('-')[0]))
        self.keyr.columns = newcols
        
    def _get_currency(self,string):
        for j in currencies:
            if ' '+j in string:
                return j        
        
    def _get_keyratio(self,string):
        output = []
        for i,j in self.keyr.iterrows():
            try:
                if string in j[0]:
#                     fac = get_factor(j[0])
                    cur = self._get_currency(j[0])
                    print 'Parameter:', string
                    print 'Currency: ', cur
                    for jj in j.keys()[1:]:
                        try:
                            val = float(j[jj].replace(',',''))
                            output.append((self.isin, jj, val))
                        except AttributeError:
                            output.append((self.isin, jj, 'NaN'))                            
                    break
            except TypeError:
                continue        
        return output
            

In [88]:
for j in df[df.Mnemonic=='DAI'].ISIN:
    print j

DE0007100000


In [94]:
df[df.Mnemonic=='DAI'].ISIN.values[0]

'DE0007100000'

In [95]:
try:
    s = keyratios('DAI',directory='ms_keyratios/',isin='DE0007100000')
except IOError:
    pass

In [96]:
s.keyr

,0,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Revenue EUR Mil,"99,222","99,555","98,469",NaN,"97,761","106,540","114,297","117,982","129,872","149,467","152,688"
1,Gross Margin %,20.6,24.1,21.9,NaN,23.3,24.0,22.3,21.3,21.7,21.3,20.7
2,Operating Income EUR Mil,"4,661","7,875","2,730","-1,513","7,274","8,449","8,116","10,139","10,173","12,744","12,002"
3,Operating Margin %,4.7,7.9,2.8,NaN,7.4,7.9,7.1,8.6,7.8,8.5,7.9
4,Net Income EUR Mil,"3,744","3,986","1,348","-2,640","4,498","5,667","6,428","6,842","6,962","8,424","8,184"
5,Earnings Per Share EUR,3.64,3.81,1.40,-2.63,4.28,5.31,6.02,6.40,6.51,7.87,7.64
6,Dividends EUR,NaN,NaN,NaN,NaN,NaN,1.85,2.20,2.20,2.25,2.45,3.25
7,Payout Ratio % *,NaN,NaN,NaN,NaN,NaN,39.5,42.0,32.0,32.1,33.9,42.5
8,Shares Mil,"1,029","1,046",959,"1,003","1,052","1,067","1,067","1,069","1,069","1,069","1,071"
9,Book Value Per Share * EUR,NaN,36.73,35.45,29.73,32.28,35.72,38.64,38.42,41.38,46.60,50.19


In [97]:
s._get_keyratio('Dividends')

Parameter: Dividends
Currency:  EUR


[('DE0007100000', 2006, 'NaN'),
 ('DE0007100000', 2007, 'NaN'),
 ('DE0007100000', 2008, 'NaN'),
 ('DE0007100000', 2009, 'NaN'),
 ('DE0007100000', 2010, 'NaN'),
 ('DE0007100000', 2011, 1.85),
 ('DE0007100000', 2012, 2.2),
 ('DE0007100000', 2013, 2.2),
 ('DE0007100000', 2014, 2.25),
 ('DE0007100000', 2015, 2.45),
 ('DE0007100000', 2016, 3.25)]

In [75]:
get_currency('Mil EUR')

'EUR'

In [ ]:
conn = sq.connect('database/stockDB.db')
c    = conn.cursor()

c.execute('''CREATE TABLE IF NOT EXISTS stocks 
    (
    NAME    TEXT,
    ISIN    CHAR(12) PRIMARY KEY,
    TICKER  TEXT
    );''')

c.executemany('INSERT INTO stocks VALUES (?,?,?)', ctuples)

# c.executemany('INSERT INTO stocks (`NAME`, `ISIN`, `TICKER`) VALUES (?,?,?) ON DUPLICATE KEY UPDATE `price` = 22.5;')

conn.commit()
conn.close()